In [2]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [16]:
tf.test.is_gpu_available()

False

In [6]:
batchsize=8

In [7]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\Sanjay Srivastava\Drowsiness Detection\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\Sanjay Srivastava\Drowsiness Detection\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 32128 images belonging to 2 classes.
Found 8031 images belonging to 2 classes.


In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r'C:\Users\Sanjay Srivastava\Drowsiness Detection\Prepared_Data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 4400 images belonging to 2 classes.


In [20]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [21]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 39, 39, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 39, 39, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 39, 39, 32)   0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

activation_166 (Activation)     (None, 3, 3, 192)    0           batch_normalization_166[0][0]    
__________________________________________________________________________________________________
conv2d_167 (Conv2D)             (None, 3, 3, 192)    258048      activation_166[0][0]             
__________________________________________________________________________________________________
batch_normalization_167 (BatchN (None, 3, 3, 192)    576         conv2d_167[0][0]                 
__________________________________________________________________________________________________
activation_167 (Activation)     (None, 3, 3, 192)    0           batch_normalization_167[0][0]    
__________________________________________________________________________________________________
conv2d_164 (Conv2D)             (None, 3, 3, 192)    147456      mixed7[0][0]                     
__________________________________________________________________________________________________
conv2d_168

In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [23]:
checkpoint = ModelCheckpoint(r'C:\Users\Sanjay Srivastava\Drowsiness Detection\models',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

Epoch 1/5
4016/4016 [==============================] - 2194s 534ms/step - loss: 0.1870 - accuracy: 0.9290 - val_loss: 0.4643 - val_accuracy: 0.7813

Epoch 00001: val_loss improved from inf to 0.46431, saving model to C:\Users\Sanjay Srivastava\Drowsiness Detection\models
INFO:tensorflow:Assets written to: C:\Users\Sanjay Srivastava\Drowsiness Detection\models\assets
Epoch 2/5
4016/4016 [==============================] - 3659s 910ms/step - loss: 0.1478 - accuracy: 0.9451 - val_loss: 0.4367 - val_accuracy: 0.7917

Epoch 00002: val_loss improved from 0.46431 to 0.43672, saving model to C:\Users\Sanjay Srivastava\Drowsiness Detection\models
INFO:tensorflow:Assets written to: C:\Users\Sanjay Srivastava\Drowsiness Detection\models\assets
Epoch 3/5
4016/4016 [==============================] - 4406s 1s/step - loss: 0.1380 - accuracy: 0.9481 - val_loss: 0.4188 - val_accuracy: 0.7967

Epoch 00003: val_loss improved from 0.43672 to 0.41881, saving model to C:\Users\Sanjay Srivastava\Drowsiness De

In [3]:
model=load_model(r'C:\Users\Sanjay Srivastava\Drowsiness Detection\models')

In [8]:
acc_tr, loss_tr = model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

4016/4016 [==============================] - 1412s 348ms/step - loss: 0.1073 - accuracy: 0.9591
0.10725124925374985
0.959132194519043


In [9]:
acc_vr, loss_vr = model.evaluate(validation_data)
print(acc_vr)
print(loss_vr)

1004/1004 [==============================] - 324s 323ms/step - loss: 0.4056 - accuracy: 0.8278
0.40563786029815674
0.8277922868728638


In [11]:
acc_test, loss_test = model.evaluate(test_data)
print(acc_tr)
print(loss_tr)

550/550 [==============================] - 168s 305ms/step - loss: 0.8729 - accuracy: 0.8014
0.10725124925374985
0.959132194519043
